<a href="https://colab.research.google.com/github/anandaditya07/Hate-Speech/blob/main/ml_lab02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1: Install & Import Libraries**

In [24]:
!pip install -q scikit-learn pandas numpy emoji


In [25]:
import pandas as pd
import numpy as np
import re
import emoji

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, f1_score


**2: Load Dataset**

In [26]:
import pandas as pd

train_df = pd.read_csv("/content/drive/MyDrive/SubTask-B-train.csv", encoding='latin-1')
test_df = pd.read_csv("/content/drive/MyDrive/SubTask-B-test.csv", encoding='latin-1')

train_df.head()  #used for previewing the dataset

print("Raw Dataset Shape:", train_df.shape)
train_df[['tweet', 'label']].head()

Raw Dataset Shape: (19019, 2)


,tweet,label
0,à¤ à¤®à¤° à¤à¤à¤¾à¤²à¤¾ à¤¦à¥à¤¨à¤¿à¤ à¤¸à...,0
1,à¤¬à¥à¤à¥à¤ªà¥ @BJP4India à¤à¥à¤¨à¥à¤¦à...,0
2,#AssemblyElections2022 à¤ªà¥à¤°à¤­à¤¾à¤°à¥ à...,0
3,#à¤¬à¥à¤°à¥à¤à¤¿à¤à¤ - à¤®à¤§à¥à¤¯à¤ªà¥...,0
4,#AmitShah à¤¨à¥ à¤à¤¹à¤¾ à¤à¤¤à¥à¤¤à¤° à¤ª...,0


**3:Text Cleaning Function**

In [27]:
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r"http\S+|www\S+", "", text)
    text = re.sub(r"@\w+|#\w+", "", text)
    text = emoji.replace_emoji(text, replace="")
    text = re.sub(r"[^a-zA-Z\s]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text


**4: Apply Cleaning**

In [28]:
train_df['clean_text'] = train_df['tweet'].apply(clean_text)

# Remove empty rows
train_df = train_df[train_df['clean_text'] != ""]
train_df.reset_index(drop=True, inplace=True)

train_df[['tweet', 'clean_text']].head()

,tweet,clean_text
0,LIVE | à¤®à¤¹à¤à¤à¤¾à¤ à¤ªà¤° à¤ªà¤¡à¤¼à¥ ...,live
1,#OpinionPoll : à¤¯à¥à¤à¥ à¤à¥ à¤à¥à¤¨à¤...,live at
2,#ElectionWithNN : 'à¤ªà¤¶à¥à¤à¤¿à¤®à¥ à¤à¤...,round voting update live more updates
3,Goa Election 2022 | à¤à¥à¤µà¤¾ à¤®à¥à¤ à¤...,goa election
4,UP Election 2022: à¤¨à¥à¤ªà¤¾à¤² à¤¬à¤¾à¤°à¥...,up election


In [29]:
test_df['clean_text'] = test_df['text'].apply(clean_text)
test_df = test_df[test_df['clean_text'] != ""]
test_df.reset_index(drop=True, inplace=True)

print("\nAfter Removing Empty Rows from test_df:", test_df.shape)


After Removing Empty Rows from test_df: (674, 3)


**5: Train–Validation Split**

In [30]:
X_train, X_val, y_train, y_val = train_test_split(
    train_df['clean_text'],
    train_df['label'],
    test_size=0.2,
    random_state=42,
    stratify=train_df['label']
)

print("Train size:", len(X_train))
print("Validation size:", len(X_val))

Train size: 2522
Validation size: 631


**6: TF-IDF Vectorization**

In [31]:
tfidf = TfidfVectorizer(
    max_features=5000,
    ngram_range=(1, 2)
)

X_train_tfidf = tfidf.fit_transform(X_train)
X_val_tfidf = tfidf.transform(X_val)

print("\nTF-IDF Shape (Train):", X_train_tfidf.shape)
print("Sample TF-IDF Vector:\n", X_train_tfidf[0].toarray())



TF-IDF Shape (Train): (2522, 3814)
Sample TF-IDF Vector:
 [[0. 0. 0. ... 0. 0. 0.]]


**7: Train Logistic Regression Model**

In [32]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train_tfidf, y_train)


LogisticRegression(max_iter=1000)

**8: Evaluation**

In [33]:
y_pred = model.predict(X_val_tfidf)

print("Classification Report:\n")
print(classification_report(y_val, y_pred))

print("Macro F1 Score:", f1_score(y_val, y_pred, average="macro"))


Classification Report:

              precision    recall  f1-score   support

           0       0.90      1.00      0.95       570
           1       0.00      0.00      0.00        61

    accuracy                           0.90       631
   macro avg       0.45      0.50      0.47       631
weighted avg       0.82      0.90      0.86       631

Macro F1 Score: 0.4746044962531224


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


**9: Predict on Test Data**

In [34]:
test_df['clean_text'] = test_df['text'].apply(clean_text)

test_tfidf = tfidf.transform(test_df['clean_text'])
test_df['prediction'] = model.predict(test_tfidf)

test_df.to_csv("logistic_predictions.csv", index=False)
print("Predictions saved")

Predictions saved


**Deep Learning Code - BiLSTM**

**1: Install & Import**

In [35]:
!pip install -q tensorflow keras emoji pandas scikit-learn


In [36]:
import pandas as pd
import re
import emoji
import numpy as np

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense
from tensorflow.keras.callbacks import EarlyStopping

**2: Clean Data**

In [37]:
import pandas as pd
import re
import emoji

# Sample dataframe (to show output clearly)
data = {
    "text": [
        "I HATE you!!! 😡😡",
        "Visit https://abc.com now",
        "@user This is GOOD #happy",
        "😡😡😡",
        "   "
    ]
}

df = pd.DataFrame(data)

print("🔹 Original Data:")
print(df)

# Preprocessing function (YOUR logic)
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r"http\S+|www\S+", "", text)
    text = re.sub(r"@\w+|#\w+", "", text)
    text = emoji.replace_emoji(text, replace="")
    text = re.sub(r"[^a-zA-Z\s]", "", text)
    return text.strip()

# Apply preprocessing
df['clean_text'] = df['text'].apply(clean_text)

print("\n🔹 After Cleaning:")
print(df[['text', 'clean_text']])

# Remove empty rows
df = df[df['clean_text'] != ""]

print("\n🔹 After Removing Empty Rows:")
print(df)


🔹 Original Data:
                        text
0           I HATE you!!! 😡😡
1  Visit https://abc.com now
2  @user This is GOOD #happy
3                        😡😡😡
4                           

🔹 After Cleaning:
                        text    clean_text
0           I HATE you!!! 😡😡    i hate you
1  Visit https://abc.com now    visit  now
2  @user This is GOOD #happy  this is good
3                        😡😡😡              
4                                         

🔹 After Removing Empty Rows:
                        text    clean_text
0           I HATE you!!! 😡😡    i hate you
1  Visit https://abc.com now    visit  now
2  @user This is GOOD #happy  this is good


**3: Tokenization**

In [38]:
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(train_df['clean_text'])

sequences = tokenizer.texts_to_sequences(train_df['clean_text'])
padded = pad_sequences(sequences, maxlen=100, padding="post")

**4: Train–Validation Split**

In [39]:
X_train, X_val, y_train, y_val = train_test_split(
    padded,
    train_df['label'],
    test_size=0.2,
    random_state=42,
    stratify=train_df['label']
)

**5: Build BiLSTM Model**

In [40]:
model = Sequential([
    Embedding(input_dim=10000, output_dim=128, input_length=100),
    Bidirectional(LSTM(64)),
    Dense(1, activation='sigmoid')
])

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

**6: Train Model**

In [41]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=5,
    batch_size=32,
    callbacks=[EarlyStopping(patience=2)],
    verbose=1
)

Epoch 1/5
79/79 ━━━━━━━━━━━━━━━━━━━━ 31s 288ms/step - accuracy: 0.8481 - loss: 0.4011 - val_accuracy: 0.9033 - val_loss: 0.3180
Epoch 2/5
79/79 ━━━━━━━━━━━━━━━━━━━━ 27s 116ms/step - accuracy: 0.8972 - loss: 0.3285 - val_accuracy: 0.9033 - val_loss: 0.3067
Epoch 3/5
79/79 ━━━━━━━━━━━━━━━━━━━━ 11s 131ms/step - accuracy: 0.9015 - loss: 0.2862 - val_accuracy: 0.9033 - val_loss: 0.3290
Epoch 4/5
79/79 ━━━━━━━━━━━━━━━━━━━━ 21s 136ms/step - accuracy: 0.9026 - loss: 0.2485 - val_accuracy: 0.8906 - val_loss: 0.3519


**7: Evaluation**

In [42]:
val_preds = (model.predict(X_val) > 0.5).astype(int)

from sklearn.metrics import classification_report, f1_score

print(classification_report(y_val, val_preds))
print("Macro F1:", f1_score(y_val, val_preds, average="macro"))


20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 63ms/step
              precision    recall  f1-score   support

           0       0.90      0.98      0.94       570
           1       0.10      0.02      0.03        61

    accuracy                           0.89       631
   macro avg       0.50      0.50      0.49       631
weighted avg       0.83      0.89      0.85       631

Macro F1: 0.48511725263419303
